In [14]:
# Lessons Learned 
# =========================================
# 1/ when getting errors, do Cell->Run All , so they may go away
# 2/ It seems when using Def functions instead of Flat code with out Defs , we are getting errors
# 3/ But Def will help to use local variable names with out global scope , can use same var names  df etc...

In [69]:
#import desired packages
import numpy as np
import pandas as pd
import traceback
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [70]:
import requests
def extract_target_price(ticker):
    url = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/"
    url = url + ticker + "?formatted=true&crumb=RqWcRwtueFe&lang=en-US&region=US&modules=upgradeDowngradeHistory%2CrecommendationTrend%2CfinancialData%2CearningsHistory%2CearningsTrend%2CindustryTrend&corsDomain=finance.yahoo.com"
    #print(url)
    try:
        resp = requests.get(url)
        return resp.json()
    except: 
        traceback.print_exc()
        pprint("Requests ERROR for ticker:" + ticker)
        return False
    

In [71]:
import time

def get_target_price_record(ticker): 
    json_data = extract_target_price(ticker)
    #pprint(resp)
    s2 = json_data['quoteSummary']['result'][0] # there is a List( []), so get 0 th element
    s = s2['financialData']

    # Analyst Traget Prices
    current = float(s['currentPrice']['fmt'])
    low = float(s['targetLowPrice']['fmt'])
    median = float(s['targetMedianPrice']['fmt'])
    mean = float(s['targetMeanPrice']['fmt'])
    hi = float(s['targetHighPrice']['fmt'])

    today = time.strftime("%Y-%m-%d")
    target_price_rec = [ticker, today, current, low, median, mean, hi]
    return target_price_rec


In [77]:
def create_target_price_masterfile():

    #import SnP Symbol list 
    snp_tickers_allyears = pd.read_csv('data-SnP-10years-Tickers-byyear-dfready.csv', header=0)

    tprice_lst = []
    nodata_found_lst = []

    year = 2015 
    llst = snp_tickers_allyears.query('Year == @year').values.tolist()
    #llst = [ ['Ticker', 'Year'], ['FIT', 2008]]  # use this for Testing one ROW
    
    for stock in llst[1:] :  # skip 0 th element being it header
        # here index are : 0 - TIcker , 1 - year
        #print('trying to fetch data for ' + stock[0])
        try:
            rec = get_target_price_record(stock[0])  
            if len(rec) > 0:
                tprice_lst.append(rec)
                print( rec + ' good')
            else:
                print(stock[0])
                nodata_found_lst.append(stock)
        except:
            nodata_found_lst.append(stock)
            #print('Exception for { }, year {}'.format(stock[0], stock[1])

    df = pd.DataFrame(tprice_lst)
    cols = ['Ticker', 'Date', 'Current', 'Low_tp', 'Median_tp', 'Mean_tp', 'High_tp']
    df.columns = cols
    df['Below_median'] = round( ((df['Current'] - df['Median_tp']) * 100 )/df['Current'], 1) 
    df['Below_mean'] = round( ((df['Current'] - df['Mean_tp']) * 100 )/df['Current'], 1) 
    df2 = df.sort_values(by='Below_median', ascending=True)
    df2.to_csv('data-analyst-target-price.csv', index=False)

    nodata_df = pd.DataFrame(nodata_found_lst)
    nodata_df.columns = ['Ticker', 'Year']
    nodata_df.to_csv('data-nodata-found-target-price.csv', index=False)


In [78]:
create_target_price_masterfile()

In [82]:
df3 = pd.read_csv('data-analyst-target-price.csv', header=0)
df3['Below_median'].head()
df3['b_low'] = round( ((df3['Current'] - df3['Low_tp']) * 100 )/df3['Current'], 1)
df3['a_high'] = round( ((df3['Current'] - df3['High_tp']) * 100 )/df3['Current'], 1)
df3

,Ticker,Date,Current,Low_tp,Median_tp,Mean_tp,High_tp,Below_median,Below_mean,b_low,a_high
0,DTV,2017-01-18,53.04,84.00,95.00,93.46,98.00,-79.1,-76.2,-58.4,-84.8
1,UA,2017-01-18,25.35,44.00,44.00,44.00,44.00,-73.6,-73.6,-73.6,-73.6
2,MNK,2017-01-18,46.53,64.00,78.00,81.40,108.00,-67.6,-74.9,-37.5,-132.1
3,XEL,2017-01-18,41.33,61.12,61.12,61.12,61.12,-47.9,-47.9,-47.9,-47.9
4,FTR,2017-01-18,3.54,3.00,5.00,4.87,7.50,-41.2,-37.6,15.3,-111.9
5,RRC,2017-01-18,34.35,28.00,48.00,47.31,62.00,-39.7,-37.7,18.5,-80.5
6,EQT,2017-01-18,62.53,70.00,85.00,83.29,97.00,-35.9,-33.2,-11.9,-55.1
7,QEP,2017-01-18,18.10,20.00,24.50,25.22,33.00,-35.4,-39.3,-10.5,-82.3
8,MYL,2017-01-18,37.19,43.00,50.00,50.11,57.00,-34.4,-34.7,-15.6,-53.3
9,SWN,2017-01-18,9.69,7.00,13.00,13.61,20.00,-34.2,-40.5,27.8,-106.4


In [84]:
import urllib.request

# source url from here http://stackoverflow.com/questions/25338608/download-all-stock-symbol-list-of-a-marketdd
ul = "http://www.nasdaq.com/screening/companies-by-name.aspx?A&pagesize=200&render=download"

urllib.request.urlretrieve (ul, "nasdaq.csv")


('nasdaq.csv', <http.client.HTTPMessage at 0x7efcd20fcc88>)

In [124]:

dfs = pd.read_csv('nasdaq.csv', header=0)

header = "Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote, "
cols = header.split(",")

dfs.columns = cols
dfs.head()
#dfs['Mcap2']=dfs['Marketcap'].apply(lambda x:float(x[1:-2])*(1000 if (x[-1] =='B') else 1)
                                    
                                    

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,
0,PIH,"1347 Property Insurance Holdings, Inc.",7.2,$42.9M,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN
1,FLWS,"1-800 FLOWERS.COM, Inc.",10.1,$659.35M,1999,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws,NaN
2,FCCY,1st Constitution Bancorp (NJ),16.7,$133.32M,n/a,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy,NaN
3,SRCE,1st Source Corporation,43.7,$1.13B,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN
4,VNET,"21Vianet Group, Inc.",7.09,$806.01M,2011,Technology,"Computer Software: Programming, Data Processing",http://www.nasdaq.com/symbol/vnet,NaN


In [126]:
from bs4 import BeautifulSoup

def create_zack_ranks():
    zack_list = []
    
    for ticker in ["FIT", "EBAY"]:
        url = "https://www.zacks.com/stock/quote/{}?q={}".format(ticker, ticker)
        resp = requests.get(url)
        html_doc = resp.text

        soup = BeautifulSoup(html_doc, 'html.parser')

        today = time.strftime("%Y-%m-%d")
        rec = [ticker, today]

        # ref: http://stackoverflow.com/questions/22217713/how-to-select-a-class-of-div-inside-of-a-div-with-beautiful-soup
        for rcr in  soup.find_all('div',  attrs={'class': 'rank_container_right'}):
            cg = rcr.find('div', attrs={'class': 'composite_group'})
            for spn in cg.p.find_all('span'):
                #print(spn.text)
                rec.append(spn.text)
                
        zack_list.append(rec)
            
    df = pd.DataFrame(zack_list)
    cols = ['Ticker', 'Date', 'Value', 'Growth', 'Momentum', 'VGM']
    df.columns = cols      
    print(df)
    

In [127]:
create_zack_ranks()

  Ticker        Date Value Growth Momentum VGM
0    FIT  2017-01-19     A      A        D   A
1   EBAY  2017-01-19     C      F        F   D
